In [1]:
%runfile testcong.py

## Read all optimal curves from the internal database (conductors from $1$ to $500000$) and make a hash table from the $a_p$ for $50$ primes (the first $50$ over $500000$).  This takes about 47 minutes so after doing it once we save the result, and later runs will read this (which takes a few seconds: the saved file is 52M).

In [2]:
try:
    hashtab_13_50 = load('hashtab13_50')
except IOError:
    hashtab_13_50 = make_hash(13,1,500000,50)
    hashtab_13_50 = dict([(k,v) for k,v in hashtab_13_50.items() if len(v)>1])
    save(hashtab_13_50, 'hashtab13_50')
len(hashtab_13_50)

177

## The hash table is a dictionary: congruent curves will have the same hash (but possibly some incongruent curves will also have the same hash, if they have the same values of $a_p\pmod{13}$ for all $50$ primes tested).  So we extract sets of mutually congruent curves by looking at the dictionary's values (which are lists of labels) of length greater than one.

In [3]:
congruent_sets = [s for s in hashtab_13_50.values() if len(s)>1]
print("The number of nontrivial congruent sets is {}".format(len(congruent_sets)))
from collections import Counter
sizes = Counter()
for s in congruent_sets:
    sizes[len(s)] += 1
print("Distribution of sizes of nontrivial congruent sets: {}".format(sizes))

The number of nontrivial congruent sets is 177
Distribution of sizes of nontrivial congruent sets: Counter({2: 177})


### Test each of these pairs to see if we have an actual congruence, using Kraus's criterion (possibly after twisting to reduce the conductors).  With the default mumax=5000000 there is one pair for which this is not rigorous:  '49a1' and '78400gw1'.

In [4]:
bad_pairs = []
for s in congruent_sets:
    E1 = EllipticCurve(s[0])
    for r in s[1:]:
        E2 = EllipticCurve(r)
        res, info = test_cong(13,E1,E2, mumax=15000000, verbose=False)
        if not res:
            report(res,info,13,s[0],r)
            bad_pairs.append([s[0],r])
if bad_pairs:
    print("{} pairs of curves are not actually 13-congruent: {}".format(len(bad_pairs),bad_pairs))
else:
    print("all {} sets of curves really are 13-congruent".format(len(congruent_sets)))

all 177 sets of curves really are 13-congruent


### So we have no "false positive" pairs

In [5]:
Set([Set([EllipticCurve(lab).j_invariant() for lab in s]) for s in congruent_sets])

{{229376/675, 633814853024541310976/367993254509587395}, {-192843857539240787968/11336014217619, -462482914449031168/3326427}, {-262144/1035, 125147927114815865709295304704/64514985611316331088611125}, {-5714497/578, 520485930308291137904991956163697/10324609571495512997764268032}, {432/169, -328568038616615609088/546688785009341767}, {-138357846491853121383730987168838623/55816105091607428996184145920, 16974593/42750}, {-15298178/3125, 229651351304189696/172613560719655}, {-2673465150439/6656, -4588389636756983840791/348165727012709497424}, {-3712000/3, 1375088009512735250/59049}, {-328509/7, 16243931652855856371/14840880223020397}, {442368/13, -328568038616615609088/546688785009341767}, {-338463151209/3731840000, -8466080937320973893386418208940863778835049/3287760130713485443760036835295043584000}, {-27476276164729743854449/10604287663073280, -334350814467169/16500}, {5831/5, 172032746578729129/60555631504375}, {-79370312059129/12960, 4307133670770643495402925929/42379253152021800},

In [6]:
len(_)

19

### Now we reduce to a smaller set of pairs such that every congruent pair is a twist of one in the smaller set

In [7]:
j_pairs = [Set([EllipticCurve(s).j_invariant() for s in s2]) for s2 in congruent_sets]
j_sets = list(Set(j_pairs))
print("Out of {} pairs, there are only {} distinct pairs of j-invariants".format(len(j_pairs),len(j_sets)))

Out of 177 pairs, there are only 19 distinct pairs of j-invariants


In [8]:
congruent_pairs = []
for jset in j_sets:
    for s2 in congruent_sets:
        if jset==Set([EllipticCurve(s).j_invariant() for s in s2]):
            congruent_pairs.append(s2)
            break
print("{} congruent pairs up to twist: {}".format(len(congruent_pairs),congruent_pairs))

19 congruent pairs up to twist: [['11025q1', '143325dp1'], ['14157s1', '99099z1'], ['1725a1', '50025c1'], ['12274c1', '135014s1'], ['52a1', '988b1'], ['418950m1', '418950n1'], ['237160f1', '237160g1'], ['1274h1', '21658t1'], ['161472k1', '161472l1'], ['14175w1', '184275ca1'], ['7488bb1', '142272fd1'], ['1190a1', '265370d1'], ['286110cy1', '286110cz1'], ['70805j1', '70805k1'], ['5070j1', '35490bg1'], ['11638o1', '151294h1'], ['11970o1', '131670ek1'], ['490050bz1', '490050cr1'], ['314330o1', '314330p1']]


### Test whether all these are irreducible, i.e. have no $13$-isogenies:

In [9]:
for s2 in congruent_pairs:
    for s in s2:
        E = EllipticCurve(s)
        assert not E.isogenies_prime_degree(13)

#### In fact the only isogeny degrees among these curves are 2, 3:

In [10]:
sum([Set(EllipticCurve(s[0]).reducible_primes()) for s in congruent_pairs],Set())

{2, 3}

In [11]:
out = open("mod13pairs.m",'w')
out.write('pairs := [\\\n')
for s in congruent_sets[:-1]:
    out.write('["{}", "{}"],\\\n'.format(s[0],s[1]))
s = congruent_sets[-1]
out.write('["{}", "{}"]\\\n'.format(s[0],s[1]))
out.write('];\n')
out.close()

In [12]:
out = open("mod13pairs_uptotwist.m",'w')
out.write('pairs := [\\\n')
for s in congruent_pairs[:-1]:
    out.write('[EllipticCurve("{}"), EllipticCurve("{}")],\\\n'.format(s[0],s[1]))
s = congruent_pairs[-1]
out.write('[EllipticCurve("{}"), EllipticCurve("{}")]\\\n'.format(s[0],s[1]))
out.write('];\n')
out.close()

In [13]:
len(congruent_pairs)

19

In [15]:
for pair in congruent_sets:
    sizes = [len(EllipticCurve(lab).isogeny_class()) for lab in pair]
    if sizes != [1,1]:
        print("pair {}: class sizes {}".format(pair,sizes))

pair ['7488bb1', '142272fd1']: class sizes [2, 1]
pair ['11025q1', '143325dp1']: class sizes [2, 1]
pair ['11760bk1', '152880fi1']: class sizes [2, 1]
pair ['10816ba1', '205504r1']: class sizes [2, 1]
pair ['832d1', '15808g1']: class sizes [2, 1]
pair ['35280ev1', '458640lf1']: class sizes [2, 1]
pair ['20800cg1', '395200dh1']: class sizes [2, 1]
pair ['24336br1', '462384ek1']: class sizes [2, 1]
pair ['11760cn1', '152880r1']: class sizes [2, 1]
pair ['3675i1', '47775bz1']: class sizes [2, 1]
pair ['25168be1', '478192bm1']: class sizes [2, 1]
pair ['300352bj1', '300352bk1']: class sizes [1, 2]
pair ['244036i1', '244036j1']: class sizes [1, 2]
pair ['2548i1', '48412k1']: class sizes [2, 1]
pair ['124215a1', '124215b1']: class sizes [2, 1]
pair ['10816c1', '205504ch1']: class sizes [2, 1]
pair ['832h1', '15808u1']: class sizes [2, 1]
pair ['11025u1', '143325dg1']: class sizes [2, 1]
pair ['2205f1', '28665z1']: class sizes [2, 1]
pair ['735c1', '9555h1']: class sizes [2, 1]
pair ['372645d

In [17]:
[legendre_symbol(a,13) for a in [2,3]]

[-1, 1]

In [18]:
npc=0
out = open("mod13_pairs.m", "w")
out.write('pairs := [\\\n')
for i,pair in enumerate(congruent_pairs):
    cc = [all_c(c) for c in pair]
    for c1 in cc[0]:
        for c2 in cc[1]:
            npc+=1
            out.write('["{}", "{}"]'.format(c1[0],c2[0]))
            if i+1<len(congruent_pairs):
                out.write(',')
            out.write('\\\n')
out.write('];\n')
out.close()           
print("{} pairs written to file".format(npc))            

23 pairs written to file


In [19]:
len(congruent_pairs)

19

In [16]:
def class_js(lab):
    E = EllipticCurve(lab)
    return Set([E1.j_invariant() for E1 in E.isogeny_class()])
def set_js(s):
    return sum([class_js(lab) for lab in s], Set())

In [17]:
irred_js = Set()
for s in congruent_sets:
    js = set_js(s)
    irred_js = irred_js + Set(js)

In [18]:
len(irred_js)

39

In [19]:
len(congruent_sets)

177

In [15]:
from sage.schemes.elliptic_curves.cm import is_cm_j_invariant
is_cm_j_invariant(8001)

(False, None)

In [17]:
[j for j in irred_js if is_cm_j_invariant(j)[0]]

[]